# Transcription Factor Classifier

In [1]:
from nn import NeuralNetwork
import numpy as np
from typing import List, Dict, Tuple, Union
from numpy.typing import ArrayLike
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import pandas as pd
from TFC_io import read_text_file, read_fasta_file
from preprocess import sample_seqs, one_hot_encode_seqs, correct_length
import matplotlib.pyplot as plt

### Read in Rap1 motif examples

In [2]:
pos_seqs = read_text_file("../data/rap1-lieb-positives.txt")
pos_labels = [True for i in range(len(pos_seqs))] #generate labels, where True = positive

### Read in negative example from yeast

In [3]:
neg_seqs = read_fasta_file("../data/yeast-upstream-1k-negative.fa")

### Correct lengths of longer negative sequences by splitting them up into multiple sequences

In [4]:
yeast_neg_corrected = correct_length(pos_seqs, neg_seqs)
print("Number of negative sequences: ", len(yeast_neg_corrected))
print("Number of positive sequences: ", len(pos_seqs))
neg_labels = [False for i in range(len(yeast_neg_corrected))]

Number of negative sequences:  183297
Number of positive sequences:  137


### Explain Sampling Scheme
I chose to use oversampling to correct the class imbalance in the data by resampling the positive class with replacement until it had the same number of observations as the negative class. While this can make the classification model prone to ovefitting, it prevents information loss and avoids an extremely small sample size if we were to downsample the negative sequences to the size of the pos_seqs.

### Implement sampling scheme

In [5]:
seqs = pos_seqs + yeast_neg_corrected #combine the two lists of seqs
labels = pos_labels + neg_labels #combine the two lists of labels
new_seqs, new_labels = sample_seqs(seqs, labels) #fix class imbalance
print(len(new_seqs)) #should be 36594
print(len(new_labels)) #should be 36594

366594
366594


### Generate training, validation, and held out sets

In [6]:
#use a 70/30 train/test split, and hold out 10% of the data for later accuracy assessment
train_val_x, held_out_x, train_val_y, held_out_y = train_test_split(new_seqs, new_labels, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(train_val_x, train_val_y, test_size=0.3, random_state = 42)

print(len(held_out_x))
print(len(X_train))
print(len(X_val))

36660
230953
98981


### One hot encode the sequences in the training, test, and validation sets

In [7]:
held_out_encoded = one_hot_encode_seqs(held_out_x)
X_train_encoded = one_hot_encode_seqs(X_train)
X_val_encoded = one_hot_encode_seqs(X_val)

In [8]:
#make sure to expand the dimensions of the expected outputs and format as numpy arrays
y_train = np.array(np.expand_dims(np.array(y_train),axis=1))
y_val = np.array(np.expand_dims(np.array(y_val),axis=1))

### Define plotting function so loss history of neural network can be visualized

In [9]:
def plot_loss_history(per_epoch_loss_train, per_epoch_loss_val):
    """
    Plots the loss history after training is complete.
    """
    loss_hist = per_epoch_loss_train
    loss_hist_val = per_epoch_loss_val
    assert len(loss_hist) > 0, "Need to run training before plotting loss history"
    fig, axs = plt.subplots(2, figsize=(8,8))
    fig.suptitle('Loss History')
    axs[0].plot(np.arange(len(loss_hist)), loss_hist)
    axs[0].set_title('Training Loss')
    axs[1].plot(np.arange(len(loss_hist_val)), loss_hist_val)
    axs[1].set_title('Validation Loss')
    plt.xlabel('Steps')
    axs[0].set_ylabel('Train Loss')
    axs[1].set_ylabel('Val Loss')
    fig.tight_layout()

### Train a neural network
I will be exploring some hyperparameters in the next few code chunks

In [10]:
nn_arch = [{'input_dim': 68, 'output_dim': 5, 'activation': 'sigmoid'},
           {'input_dim': 5, 'output_dim': 1, 'activation': 'sigmoid'}]
classifier = NeuralNetwork(nn_arch, 
                          lr=0.1, 
                          seed=42, 
                          batch_size=5, 
                          epochs=100,
                          loss_function = "binary cross entropy")

In [ ]:
#fit the neural network
per_epoch_loss_train, per_epoch_loss_val = classifier.fit(X_train_encoded, y_train, X_val_encoded, y_val)
plot_loss_history(per_epoch_loss_train, per_epoch_loss_val)
len(per_epoch_loss_train)
print(per_epoch_loss_train)

1


In [21]:
small = X_train_encoded[0:20]
small_output = y_train[0:20]
small_val = X_val_encoded[0:10]
small_val_output = y_val[0:10]

In [23]:
per_epoch_loss_train, per_epoch_loss_val = classifier.fit(small, small_output, small_val, small_val_output)
per_epoch_loss_train

1


[9.836546436824847,
 5.50011250146249,
 5.510822585372971,
 5.521373340039746,
 5.531772110056782,
 5.542025587137956,
 5.552139892430683,
 5.562120645992551,
 5.571973025764724,
 5.581701817895362,
 5.5913114598974225,
 5.600806077837153,
 5.610189518522461,
 5.619465377483181,
 5.628637023390849,
 5.637707619454434,
 5.646680142235055,
 5.6555573982481775,
 5.664342038664413,
 5.673036572366003,
 5.681643377580798,
 5.6901647122770695,
 5.698602723478958,
 5.706959455635938,
 5.715236858162651,
 5.723436792247959,
 5.731561037018711,
 5.739611295132567,
 5.74758919786393,
 5.755496309738472,
 5.763334132766012,
 5.771104110312449,
 5.778807630651145,
 5.786446030223026,
 5.794020596637796,
 5.8015325714390755,
 5.808983152658225,
 5.816373497175495,
 5.823704722907169,
 5.830977910834558,
 5.838194106888653,
 5.845354323703844,
 5.852459542251089,
 5.859510713362123,
 5.866508759152787,
 5.873454574353849,
 5.880349027558206,
 5.887192962388753,
 5.893987198595729,
 5.900732533086614

### Choice of loss function = Binary Cross Entropy
### Choice of hyperparameters: number of hidden layers = , learning rate = , batch size = , epochs = 
I chose to use binary cross entropy as my loss function because we are trying to predict a binary output, either 0 (the motif is not a binding site for Rap1) or 1 (the motif is a binding site for Rap1). I chose the hyperaparameters I did because, as shown in the above plots, they give the lowest validation loss and a relatively low training loss.

In [ ]:
#Re-run final model
nn_arch = 
final model = 
per_epoch_loss_train, per_epoch_loss_val = final_model.fit()

### Plotting training and validation loss per epoch for final model

In [ ]:
plot_loss_history(per_epoch_loss_train, per_epoch_loss_val)

In [18]:
len(rap1_motifs[0])

17

In [17]:
len(yeast_neg[1001])

1000

In [19]:
int(58.8)

58

In [20]:
temp = [1,2,3,4,5,6,7,8,9, 0, 1, 2, 3, 4, 5,6,7,8]

In [21]:
temp[0:17]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7]

In [12]:
temp = np.array([[1,2,3],
                [4,5,6],
                [7,8,9]])

In [15]:
0.00000001 + temp

array([[1.00000001, 2.00000001, 3.00000001],
       [4.00000001, 5.00000001, 6.00000001],
       [7.00000001, 8.00000001, 9.00000001]])